<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Оценка-R2-LinearRegression" data-toc-modified-id="Оценка-R2-LinearRegression-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Оценка R2 LinearRegression</a></span></li><li><span><a href="#Оценка-R2-LinearRegression-для-зашифрованных-данных" data-toc-modified-id="Оценка-R2-LinearRegression-для-зашифрованных-данных-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Оценка R2 LinearRegression для зашифрованных данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

# Защита персональных данных клиентов

Необходимо защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, а так же после преобразований качество моделей машинного обучения должны не ухудшиться.

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
data = pd.read_csv('/datasets/insurance.csv')
print(data.head())
print(data.info())
print(data.describe())


   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
               Пол      Возраст      Зарплата  Члены семьи  Страховые выплаты
count  5000.000000  5000.000000   5000.000000  5000.000000        5000.0000

### Вывод

Результаты EDA:
- данные не имеют пропусков
- все признаки количественные
- данные не имеют выбросов

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Задача:**  Признаки умножаем на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Качество линейной регрессии не изменится.



**Обоснование:**
1. Домножим матрицу признаков Х на обратимую матрицу P.
2. В формулу обучения подставим новую матрицу XP. $w' = ((XP)^T XP)^{-1} (XP)^Ty$
3. Докажим что $a=a'$, где $a' = XPw'$


Попробуем упростить выражение для w'

$$
w' = ((XP)^T XP)^{-1} (XP)^Ty
$$
$$
= (P^T X^T X P)^{-1} (P^T X^T)y
$$
$$
= (P^T (X^T X) P)^{-1} (P^T X^T)y
$$
$$
= P^{-1} (P^T (X^T X))^{-1} P^T X^Ty
$$
$$
= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^Ty
$$
$$
= \{(P^T)^{-1} P^T = E\} = P^{-1} (X^T X)^{-1} E X^Ty
$$
$$
= P^{-1} (X^T X)^{-1} X^Ty
$$
$$
= \{(X^T X)^{-1} X^Ty = w\}
$$
$$
= P^{-1}w
$$
следовательно получаем $w'= P^{-1} w$



Подставим полученную формулу обучения в формулу предсказаний для измененной матрицы XP:

$$
a' = XP(P^{-1}w)
$$
$$
a' = X(PP^{-1})w
$$
$$
a' = XEw = Xw
$$
$$
a' = a
$$

Таким образов, мы доказали, что предсказания не изменятся при умножении матрицы признаков на обратимую матрицу.

## Алгоритм преобразования

Для шифрования данных, можем использовать свойства, которые доказали в прошлом шаге:
- умножим матрицу признаков на обратимую матрицу

**Алгоритм**

1. Сохранить матрицу признаков, X
2. Найти кол-во колонок матрицы признаков, n
3. Сгенерировать квадратную матрицу P размером nxn и заполнить случайными значениями
4. Проверить матрицу P на обратимость
    - Если необратимая,то вернуться к п.3
5. Перемножить матрицу признаков на обратимую матрицу.
    - при умножении матриц важен порядок. число столбцов первой матрицы должно быть равно числу строк второй матрицы
    - умножение матрица - это скалярное произведение строк первой матрицы на столбцы второй матрицы
    - умножаем X @ P, размер получившейся матрицы будет:
        - кол-во строк = кол-во строк матрицы X,
        - кол-во столбцов  = кол-во столбцов матрицы P

In [ ]:
def GenerateInvertableMatrix(n):
    is_inv = 0

    while is_inv == 0:
        P = np.random.normal(size=(n,n))
        try:
            np.linalg.inv(P)
        except:
            continue

        is_inv = 1

    return P


In [ ]:
def EncryptFeatures(features):
    X = features.values
    return X @ GenerateInvertableMatrix(X.shape[1])

## Проверка алгоритма

In [ ]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

### Оценка R2 LinearRegression

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(r2_score(target_test, predictions))

0.43522757127026546


### Оценка R2 LinearRegression для зашифрованных данных

In [ ]:
features = EncryptFeatures(features)

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(r2_score(target_test, predictions))

0.43522757127005385


### Вывод

Оценки R2 совпали.

Умножая данные на обратимую матрицу получим одинаковый результат.